In [ ]:
!pip install pytube
!pip install moviepy

In [ ]:
import json
import requests

def get_all_video_in_channel(channel_id, api_key):
    base_video_url = 'https://www.youtube.com/watch?v='
    base_search_url = 'https://www.googleapis.com/youtube/v3/search?'

    first_url = base_search_url+'key={}&channelId={}&part=snippet,id&order=date&maxResults=25'.format(api_key, channel_id)

    video_links = []
    url = first_url
    while True:
        resp = requests.get(url)
        resp_json = resp.json()

        for i in resp_json['items']:
            if i['id']['kind'] == "youtube#video":
                video_links.append(base_video_url + i['id']['videoId'])

        try:
            next_page_token = resp_json['nextPageToken']
            url = first_url + '&pageToken={}'.format(next_page_token)
        except KeyError:
            break
    return video_links


In [ ]:
channel_id = 'UCHCZnC9akNA9pBP7aJGNKdg'
youtube_api_key = ''
videos = get_all_video_in_channel(channel_id, youtube_api_key)

In [ ]:
videos[5]

'https://www.youtube.com/watch?v=kUTfaVLPOiw'

In [ ]:
from pytube import YouTube
import moviepy.editor as mp
from tqdm import tqdm

def get_audio_from_youtube_video(url, output_file_path):
    # Create a YouTube object
    yt = YouTube(url)

    # Select the audio stream (.first() is for choosing the best stream)
    audio_stream = yt.streams.filter(only_audio=True).first()

    # Download the audio stream to a temporary .mp4 file
    audio_stream.download(filename='temporary_audio_file.mp4')

    # Convert the downloaded file to .mp3
    converted_clip = mp.AudioFileClip('temporary_audio_file.mp4')
    converted_clip.write_audiofile(output_file_path)

    # Remove the temporary .mp4 file
    converted_clip.close()

In [ ]:
i = 1
for video in tqdm(videos, desc='Downloading'):
    try:
        output_path = f'{YOUR_OUTPUT_DIRECTORY}/audio_{i}.mp3'
        download_youtube_audio(video, output_path)
        i += 1
    except:
        pass

In [ ]:
from transformers import pipeline
import re

pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

In [ ]:
import glob

# Gathering audio files for transcription
audios = glob.glob('YOUR_PATH_TO_AUDIO_FILES/*')

transcripts_dir = 'DIR_TO_TRANSRIPTS'
i = 1
for audio in tqdm(audios, desc='Processing'):
    transcript = pipe(video)
    with open(f'{transcripts_dir}/transcript_{i}.txt', 'w') as f:
        f.write(transcript['text'])
    i += 1